In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from model import FC, Discriminator, Loss_dis
from tqdm import tqdm
import os
from scipy import io
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [2]:

## data load

path = r'../DEAP/data_preprocessed_matlab/'  # 경로는 저장 파일 경로
file_list = os.listdir(path)

print("data path check")
for i in file_list:    # 확인
    print(i, end=' ')


data path check
s01.mat s15.mat s29.mat s28.mat s14.mat s16.mat s02.mat s03.mat s17.mat s13.mat s07.mat s06.mat s12.mat s04.mat s10.mat s11.mat s05.mat s20.mat s08.mat s09.mat s21.mat s23.mat s22.mat s32.mat s26.mat s27.mat s19.mat s25.mat s31.mat s30.mat s24.mat s18.mat 

In [3]:
for i in tqdm(file_list, desc="read data"): 
    mat_file = io.loadmat(path+i)
    data = mat_file['data']
    labels = np.array(mat_file['labels'])
    val = labels.T[0].round().astype(np.int8)
    aro = labels.T[1].round().astype(np.int8)
    
    if(i=="s01.mat"): 
        Data = data
        VAL = val
        ARO = aro
        continue
        
    Data = np.concatenate((Data ,data),axis=0)   # 밑으로 쌓아서 하나로 만듬
    VAL = np.concatenate((VAL ,val),axis=0)
    ARO = np.concatenate((ARO ,aro),axis=0)

read data: 100%|██████████| 32/32 [00:28<00:00,  1.11it/s]


In [4]:
print(len(VAL))
print(len(Data))

1280
1280


In [5]:
# eeg preprocessing

eeg_data = []
peripheral_data = []

for i in tqdm(range(len(Data)), desc="preprocess channel"):
    for j in range (40): 
        if(j < 32): # get channels 1 to 32
            eeg_data.append(Data[i][j])
        else:
            peripheral_data.append(Data[i][j])

# set data type, shape
eeg_data = np.reshape(eeg_data, (len(Data),1,32, 8064))
eeg_data=eeg_data.astype('float32')
eeg_data32 = torch.from_numpy(eeg_data)
VAL = (torch.from_numpy(VAL)).type(torch.long)

preprocess channel: 100%|██████████| 1280/1280 [00:00<00:00, 100267.24it/s]


In [10]:
#data 40 x 40 x 8064 video/trial x channel x data
#labels 40 x 4 video/trial x label (valence, arousal, dominance, liking)
#32명 -> 4명 / 14명 14명

# data split
print("data split")
train_data, val_data,train_label, val_label = train_test_split(eeg_data32, VAL, test_size=0.125)
x_train, x_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.5)

# make data loader
print("make data loader")
target_dataset = TensorDataset(x_train, y_train)
source_dataset = TensorDataset(x_test, y_test)
val_dataset = TensorDataset(val_data, val_label)
target_dataloader = DataLoader(target_dataset, 64, shuffle=True)
source_dataloader = DataLoader(source_dataset, 64, shuffle=True)
val_dataloader = DataLoader(val_dataset, 64, shuffle=True)

# cuda
device = torch.device("mps:0")
print("device: ", device)

#model
dis = Discriminator(15960).to(device)
fc = FC(32).to(device)

#optim
optimizer_dis = optim.SGD(dis.parameters(), lr=0.001, momentum=0.9)
optimizer_cls = optim.SGD(fc.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

data split
make data loader
device:  mps:0


In [7]:
def get_center_loss(device, features, labels, alpha, num_classes, name):

    len_features = features.size(1)

    centers = torch.zeros(num_classes, len_features).float().to(device)
    #if torch.cuda.is_available():
    #    centers = centers.cuda()

    labels = labels.view(-1, 1).long()
    centers_batch = centers.index_select(0, labels.squeeze()) # gather -> index_select
    diff = centers_batch - features
    unique_label, unique_idx, unique_count = torch.unique(labels, sorted=True, return_inverse=True, return_counts=True)
    #appear_times = unique_count.index_select(0, unique_idx)   # 일단 무시 
    #appear_times = appear_times.view(-1, 1)

    #diff = diff / (1 + appear_times.float())
    diff = alpha * diff

    centers_update_op = torch.zeros(num_classes, len_features).to(device).scatter_add_(0, labels.repeat(1, len_features), diff)
    # scatter_sub -> scatter_add_

    loss = torch.mean(torch.abs(features - centers_batch))

    return loss, centers, centers_batch, diff


In [11]:
#train
# At each iteration, we first update the parameters of the domain discriminator,\
# fix the feature extractor and classifier, and then fix the
#  domain discriminator and update the parameters of both the
#   feature extractor and classifier.

g_loss_log = []
d_loss_log = []
accuracy_s = []
accuracy_d = []
accuracy_val = []

best_loss = 10000000
limit_epoch = 9
limit_check = 0
val_loss = 0
nb_epochs = 3
for epoch in tqdm(range(nb_epochs+1)):
    temp_gloss = 0
    temp_dloss = 0
    temp_accuracy_d = 0
    temp_accuracy_s = 0
    temp_accuracy_val = 0

    print(epoch, ": epoch")

    temp = 0.0 #batch count
    fc.train()
    for i, (target, source) in enumerate(zip(target_dataloader, source_dataloader)):
        temp += 1.0
        print("batch: " , i)

        #print(i, target[0].shape, target[1].shape)
        #print(source[0].shape, source[1].shape)
        x_target, y_target = target[0], target[1]
        x_source, y_source = source[0], source[1]

        x_target = x_target.to(device)
        y_target = y_target.to(device)
        x_source = x_source.to(device)
        y_source = y_source.to(device)
        
        feat_t, pred_t = fc.forward(x_target)
        feat_s, pred_s = fc.forward(x_source)
        
        dc_t = dis(feat_t)
        dc_s = dis(feat_s)
        temp_dc_s = dc_s.clone().detach()

        #discriminators loss
        dis_loss = Loss_dis(dc_t, dc_s)
        
        for p in fc.parameters():
            p.requires_grad = False
        for p in dis.parameters():
            p.requires_grad = True

        optimizer_dis.zero_grad()
        dis_loss.backward(retain_graph=True)
        optimizer_dis.step()
    
        for p in fc.parameters():
            p.requires_grad = True   
        for p in dis.parameters():
            p.requires_grad = False

        #global loss
        g_loss_adv = torch.mean(torch.square(temp_dc_s-1))/2
        g_loss_ce_t = criterion(pred_t, y_target-1)
        g_loss_ce_s = criterion(pred_s, y_source-1)
        
        # center loss
        g_loss_center, centers, centers_batch, diff = get_center_loss(device, feat_t, y_target-1, alpha=0.5, num_classes=9, name='centers')
        #center = torch.mean(feat_t)
        #g_loss_center = torch.mean(torch.square(feat_t - centers))/2
        g_loss = g_loss_adv + g_loss_ce_s + g_loss_ce_t + g_loss_center
         
        optimizer_cls.zero_grad()
        g_loss.backward()
        optimizer_cls.step()
        
        temp_gloss += g_loss
        temp_dloss += dis_loss
        print((torch.argmax(pred_t,1)+1)[:10])
        print(y_target[:10])

        temp_accuracy_s += ((torch.argmax(pred_t,1)+1)== y_target).to(torch.float).mean()
        temp_accuracy_d += ((torch.argmax(pred_s,1)+1)== y_source).to(torch.float).mean()
    
    print("\ngloss", temp_gloss/temp)
    print("dloss", temp_dloss/temp)
    print("acc_d", temp_accuracy_d/temp)
    print("acc_s", temp_accuracy_s/temp)
    
    g_loss_log.append(temp_gloss/temp)
    d_loss_log.append(temp_dloss/temp)
    accuracy_d.append(temp_accuracy_d/temp)
    accuracy_s.append(temp_accuracy_s/temp)
    
    fc.eval()
    val_loss = 0
    batch = 0
    
    for x_val, y_val in val_dataloader:
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        _, y_pred = fc.forward(x_val)
        loss = criterion(y_pred, y_val-1)
        val_loss += loss.item()
        temp_accuracy_val += ((torch.argmax(y_pred,1)+1)== y_val).to(torch.float).mean()
        batch = batch + 1
    if val_loss > best_loss:
        limit_check += 1
        if(limit_check >= limit_epoch):
            break
    else:
        best_loss = val_loss
        limit_check = 0
        #python PROJECT\drda\drda_torch\main.py
    print("acc_val", temp_accuracy_val/batch)
    accuracy_val.append(temp_accuracy_val/batch)

print(g_loss_log)
print(d_loss_log)
print(accuracy_d)
print(accuracy_s)
print("val_loss ", val_loss)
print(accuracy_val)

torch.save(fc, './fc.pt')
torch.save(dis, './dis.pt')  

  0%|          | 0/4 [00:00<?, ?it/s]

0 : epoch
batch:  0


  0%|          | 0/4 [00:06<?, ?it/s]


NotImplementedError: The operator 'aten::_unique2' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [12]:
# print(torch.backends.mps.is_built())

True


In [13]:
# import torch

# mps_device = torch.device("mps")

# # MPS 장치에 바로 tensor를 생성합니다.
# x = torch.ones(5, device=mps_device)
# # 또는
# x = torch.ones(5, device="mps")

# # GPU 상에서 연산을 진행합니다.
# y = x * 2

# # 또는, 다른 장치와 마찬가지로 MPS로 이동할 수도 있습니다.
# model = YourFavoriteNet()  # 어떤 모델의 객체를 생성한 뒤,
# model.to(mps_device)       # MPS 장치로 이동합니다.

# # 이제 모델과 텐서를 호출하면 GPU에서 연산이 이뤄집니다.
# pred = model(x)

NameError: name 'YourFavoriteNet' is not defined